In [2]:
import pandas as pd
import requests
import csv

In [2]:
df = pd.read_csv('people_data.csv')

In [3]:
df.head()

,person,city,country,continent
0,Bertrand Piccard,Lausanne,Switzerland,Europe
1,Nick Veasey,City of Westminster,United Kingdom,Europe
2,Dan Buettner,Saint Paul,United States,Americas
3,Romulus Whitaker,New York,United States,Americas
4,Herbie Hancock,Chicago,United States,Americas


In [21]:
def get_city_country(lat, lon):
    base_url = "https://nominatim.openstreetmap.org/reverse"
    params = {
        "format": "json",
        "lat": lat,
        "lon": lon,
        "accept-language": "en"
    }

    response = requests.get(base_url, params=params)
    data = response.json()
    if 'address' in data:
        address = data['address']
        city = address.get('city', 'N/A')
        country = address.get('country', 'N/A')
        return city, country
    else:
        print("Failed to retrieve country and continent data.")
        return None, None

In [22]:
def get_continent_by_country(country_name):
    response = requests.get(f"https://restcountries.com/v2/name/{country_name}")
    if response.status_code == 200:
        country_data = response.json()[0]
        return country_data['region']
    else:
        return "Country not found or API error"

In [28]:
person_place_map = {}
for i, row in df.iterrows():
    lat, lng = row['lat'], row['lng']
    city, country = get_city_country(lat, lng)
    continent = get_continent_by_country(country)
    person_place_map[row['name']] = [city, country, continent]
    if city and country:
        continue
    else:
        print("Unable to fetch")

In [30]:
filename = "people_data.csv"
with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(['person', 'city', 'country', 'continent'])
    for person, details in person_place_map.items():
        row = [person] + details
        csvwriter.writerow(row)

In [13]:
len(df['city'].unique()), len(df['country'].unique()), len(df['continent'].unique())

(409, 102, 5)

In [14]:
df_main = pd.read_csv('cs448b-final-project/ted_main.csv')

In [15]:
df_main

,comments,description,duration,event,film_date,languages,main_speaker,name,num_speaker,published_date,ratings,related_talks,speaker_occupation,tags,title,url,views
0,4553,Sir Ken Robinson makes an entertaining and pro...,1164,TED2006,1140825600,60,Ken Robinson,Ken Robinson: Do schools kill creativity?,1,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 19645}, {...","[{'id': 865, 'hero': 'https://pe.tedcdn.com/im...",Author/educator,"['children', 'creativity', 'culture', 'dance',...",Do schools kill creativity?,https://www.ted.com/talks/ken_robinson_says_sc...,47227110
1,265,With the same humor and humanity he exuded in ...,977,TED2006,1140825600,43,Al Gore,Al Gore: Averting the climate crisis,1,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 544}, {'i...","[{'id': 243, 'hero': 'https://pe.tedcdn.com/im...",Climate advocate,"['alternative energy', 'cars', 'climate change...",Averting the climate crisis,https://www.ted.com/talks/al_gore_on_averting_...,3200520
2,124,New York Times columnist David Pogue takes aim...,1286,TED2006,1140739200,26,David Pogue,David Pogue: Simplicity sells,1,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 964}, {'i...","[{'id': 1725, 'hero': 'https://pe.tedcdn.com/i...",Technology columnist,"['computers', 'entertainment', 'interface desi...",Simplicity sells,https://www.ted.com/talks/david_pogue_says_sim...,1636292
3,200,"In an emotionally charged talk, MacArthur-winn...",1116,TED2006,1140912000,35,Majora Carter,Majora Carter: Greening the ghetto,1,1151367060,"[{'id': 3, 'name': 'Courageous', 'count': 760}...","[{'id': 1041, 'hero': 'https://pe.tedcdn.com/i...",Activist for environmental justice,"['MacArthur grant', 'activism', 'business', 'c...",Greening the ghetto,https://www.ted.com/talks/majora_carter_s_tale...,1697550
4,593,You've never seen data presented like this. Wi...,1190,TED2006,1140566400,48,Hans Rosling,Hans Rosling: The best stats you've ever seen,1,1151440680,"[{'id': 9, 'name': 'Ingenious', 'count': 3202}...","[{'id': 2056, 'hero': 'https://pe.tedcdn.com/i...",Global health expert; data visionary,"['Africa', 'Asia', 'Google', 'demo', 'economic...",The best stats you've ever seen,https://www.ted.com/talks/hans_rosling_shows_t...,12005869
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2545,17,"Between 2008 and 2016, the United States depor...",476,TED2017,1496707200,4,Duarte Geraldino,Duarte Geraldino: What we're missing in the de...,1,1505851216,"[{'id': 3, 'name': 'Courageous', 'count': 24},...","[{'id': 2596, 'hero': 'https://pe.tedcdn.com/i...",Journalist,"['TED Residency', 'United States', 'community'...",What we're missing in the debate about immigra...,https://www.ted.com/talks/duarte_geraldino_wha...,450430
2546,6,How can you study Mars without a spaceship? He...,290,TED2017,1492992000,3,Armando Azua-Bustos,Armando Azua-Bustos: The most Martian place on...,1,1505919737,"[{'id': 22, 'name': 'Fascinating', 'count': 32...","[{'id': 2491, 'hero': 'https://pe.tedcdn.com/i...",Astrobiologist,"['Mars', 'South America', 'TED Fellows', 'astr...",The most Martian place on Earth,https://www.ted.com/talks/armando_azua_bustos_...,417470
2547,10,Science fiction visions of the future show us ...,651,TED2017,1492992000,1,Radhika Nagpal,Radhika Nagpal: What intelligent machines can ...,1,1506006095,"[{'id': 1, 'name': 'Beautiful', 'count': 14}, ...","[{'id': 2346, 'hero': 'https://pe.tedcdn.com/i...",Robotics engineer,"['AI', 'ants', 'fish', 'future', 'innovation',...",What intelligent machines can learn from a sch...,https://www.ted.com/talks/radhika_nagpal_what_...,375647
2548,32,In an unmissable talk about race and politics ...,1100,TEDxMileHigh,1499472000,1,Theo E.J. Wilson,Theo E.J. Wilson: A black man goes undercover ...,1,1506024042,"[{'id': 11, 'name': 'Longwinded', 'count': 3},...","[{'id': 2512, 'hero': 'https://pe.tedcdn.com/i...",Public intellectual,"['Internet', 'TEDx', 'United States', 'communi...",A black man goes undercover in the alt-right,https://www.t

In [18]:
df_joined = pd.merge(df_main, df, left_on='main_speaker', right_on='person', how='left')

In [21]:
df_joined = df_joined.drop('person', axis=1)

In [28]:
df_nona = df_joined.dropna(how='any')

In [29]:
df_nona

,comments,description,duration,event,film_date,languages,main_speaker,name,num_speaker,published_date,ratings,related_talks,speaker_occupation,tags,title,url,views,city,country,continent
0,4553,Sir Ken Robinson makes an entertaining and pro...,1164,TED2006,1140825600,60,Ken Robinson,Ken Robinson: Do schools kill creativity?,1,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 19645}, {...","[{'id': 865, 'hero': 'https://pe.tedcdn.com/im...",Author/educator,"['children', 'creativity', 'culture', 'dance',...",Do schools kill creativity?,https://www.ted.com/talks/ken_robinson_says_sc...,47227110,Liverpool,United Kingdom,Europe
1,265,With the same humor and humanity he exuded in ...,977,TED2006,1140825600,43,Al Gore,Al Gore: Averting the climate crisis,1,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 544}, {'i...","[{'id': 243, 'hero': 'https://pe.tedcdn.com/im...",Climate advocate,"['alternative energy', 'cars', 'climate change...",Averting the climate crisis,https://www.ted.com/talks/al_gore_on_averting_...,3200520,Washington,United States,Americas
3,200,"In an emotionally charged talk, MacArthur-winn...",1116,TED2006,1140912000,35,Majora Carter,Majora Carter: Greening the ghetto,1,1151367060,"[{'id': 3, 'name': 'Courageous', 'count': 760}...","[{'id': 1041, 'hero': 'https://pe.tedcdn.com/i...",Activist for environmental justice,"['MacArthur grant', 'activism', 'business', 'c...",Greening the ghetto,https://www.ted.com/talks/majora_carter_s_tale...,1697550,New York,United States,Americas
4,593,You've never seen data presented like this. Wi...,1190,TED2006,1140566400,48,Hans Rosling,Hans Rosling: The best stats you've ever seen,1,1151440680,"[{'id': 9, 'name': 'Ingenious', 'count': 3202}...","[{'id': 2056, 'hero': 'https://pe.tedcdn.com/i...",Global health expert; data visionary,"['Africa', 'Asia', 'Google', 'demo', 'economic...",The best stats you've ever seen,https://www.ted.com/talks/hans_rosling_shows_t...,12005869,Uppsala,Sweden,Europe
5,672,"Tony Robbins discusses the ""invisible forces"" ...",1305,TED2006,1138838400,36,Tony Robbins,Tony Robbins: Why we do what we do,1,1151440680,"[{'id': 7, 'name': 'Funny', 'count': 1102}, {'...","[{'id': 229, 'hero': 'https://pe.tedcdn.com/im...",Life coach; expert in leadership psychology,"['business', 'culture', 'entertainment', 'goal...",Why we do what we do,https://www.ted.com/talks/tony_robbins_asks_wh...,20685401,Los Angeles,United States,Americas
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2533,9,In a mind-bending talk that blurs the line bet...,663,TED2017,1492992000,2,Tomás Saraceno,Tomás Saraceno: Would you live in a floating ...,1,1504796048,"[{'id': 1, 'name': 'Beautiful', 'count': 29}, ...","[{'id': 1085, 'hero': 'https://pe.tedcdn.com/i...",Artist,"['TED en Español', 'art', 'creativity', 'desig...",Would you live in a floating city in the sky?,https://www.ted.com/talks/tomas_saraceno_would...,248411,San Miguel de Tucumán,Argentina,Americas
2538,7,"Each year, one in seven large corporations com...",808,TEDxZurich,1478822400,1,Alexander Wagner,Alexander Wagner: What really motivates people...,1,1505246451,"[{'id': 10, 'name': 'Inspiring', 'count': 50},...","[{'id': 1246, 'hero': 'https://pe.tedcdn.com/i...",Economist,"['TEDx', 'business', 'corruption', 'economics'...",What really motivates people to be honest in b...,https://www.ted.com/talks/alexander_wagner_wha...,621431,Berlin,Germany,Europe
2541,12,Physics doesn't just happen in a fancy lab -- ...,957,TEDxManchester,1487462400,2,Helen Czerski,Helen Czerski: The fascinating physics of ever...,1,1505419202,"[{'id': 11, 'name': 'Longwinded', 'count': 29}...","[{'id': 2491, 'hero': 'https://pe.tedcdn.com/i...",Physicist,"['TEDx', 'physics', 'play', 'science']",The fascinating physics of everyday life,https://www.ted.com/talks/helen_czerski_fun_ho...,656791,Manchester,United Kingdom,Europe
2542,3,In the century-old statues that occupy Cape To...,795,TEDGlobal 2017,1503792000,1,Sethembil

In [7]:
df_nona= pd.read_csv('cs448b-final-project/ted_speakers_birth.csv')

In [8]:
df_nona

,Unnamed: 0,comments,description,duration,event,film_date,languages,main_speaker,name,num_speaker,...,ratings,related_talks,speaker_occupation,tags,title,url,views,city,country,continent
0,0,4553,Sir Ken Robinson makes an entertaining and pro...,1164,TED2006,1140825600,60,Ken Robinson,Ken Robinson: Do schools kill creativity?,1,...,"[{'id': 7, 'name': 'Funny', 'count': 19645}, {...","[{'id': 865, 'hero': 'https://pe.tedcdn.com/im...",Author/educator,"['children', 'creativity', 'culture', 'dance',...",Do schools kill creativity?,https://www.ted.com/talks/ken_robinson_says_sc...,47227110,Liverpool,United Kingdom,Europe
1,1,265,With the same humor and humanity he exuded in ...,977,TED2006,1140825600,43,Al Gore,Al Gore: Averting the climate crisis,1,...,"[{'id': 7, 'name': 'Funny', 'count': 544}, {'i...","[{'id': 243, 'hero': 'https://pe.tedcdn.com/im...",Climate advocate,"['alternative energy', 'cars', 'climate change...",Averting the climate crisis,https://www.ted.com/talks/al_gore_on_averting_...,3200520,Washington,United States,Americas
2,3,200,"In an emotionally charged talk, MacArthur-winn...",1116,TED2006,1140912000,35,Majora Carter,Majora Carter: Greening the ghetto,1,...,"[{'id': 3, 'name': 'Courageous', 'count': 760}...","[{'id': 1041, 'hero': 'https://pe.tedcdn.com/i...",Activist for environmental justice,"['MacArthur grant', 'activism', 'business', 'c...",Greening the ghetto,https://www.ted.com/talks/majora_carter_s_tale...,1697550,New York,United States,Americas
3,4,593,You've never seen data presented like this. Wi...,1190,TED2006,1140566400,48,Hans Rosling,Hans Rosling: The best stats you've ever seen,1,...,"[{'id': 9, 'name': 'Ingenious', 'count': 3202}...","[{'id': 2056, 'hero': 'https://pe.tedcdn.com/i...",Global health expert; data visionary,"['Africa', 'Asia', 'Google', 'demo', 'economic...",The best stats you've ever seen,https://www.ted.com/talks/hans_rosling_shows_t...,12005869,Uppsala,Sweden,Europe
4,5,672,"Tony Robbins discusses the ""invisible forces"" ...",1305,TED2006,1138838400,36,Tony Robbins,Tony Robbins: Why we do what we do,1,...,"[{'id': 7, 'name': 'Funny', 'count': 1102}, {'...","[{'id': 229, 'hero': 'https://pe.tedcdn.com/im...",Life coach; expert in leadership psychology,"['business', 'culture', 'entertainment', 'goal...",Why we do what we do,https://www.ted.com/talks/tony_robbins_asks_wh...,20685401,Los Angeles,United States,Americas
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1199,2533,9,In a mind-bending talk that blurs the line bet...,663,TED2017,1492992000,2,Tomás Saraceno,Tomás Saraceno: Would you live in a floating ...,1,...,"[{'id': 1, 'name': 'Beautiful', 'count': 29}, ...","[{'id': 1085, 'hero': 'https://pe.tedcdn.com/i...",Artist,"['TED en Español', 'art', 'creativity', 'desig...",Would you live in a floating city in the sky?,https://www.ted.com/talks/tomas_saraceno_would...,248411,San Miguel de Tucumán,Argentina,Americas
1200,2538,7,"Each year, one in seven large corporations com...",808,TEDxZurich,1478822400,1,Alexander Wagner,Alexander Wagner: What really motivates people...,1,...,"[{'id': 10, 'name': 'Inspiring', 'count': 50},...","[{'id': 1246, 'hero': 'https://pe.tedcdn.com/i...",Economist,"['TEDx', 'business', 'corruption', 'economics'...",What really motivates people to be honest in b...,https://www.ted.com/talks/alexander_wagner_wha...,621431,Berlin,Germany,Europe
1201,2541,12,Physics doesn't just happen in a fancy lab -- ...,957,TEDxManchester,1487462400,2,Helen Czerski,Helen Czerski: The fascinating physics of ever...,1,...,"[{'id': 11, 'name': 'Longwinded', 'count': 29}...","[{'id': 2491, 'hero': 'https://pe.tedcdn.com/i...",Physicist,"['TEDx', 'physics', 'play', 'science']",The fascinating physics of everyday life,https://www.ted.com/talks/helen_czerski_fun_ho...,656791,Manchester,United Kingdom,Europe
1202,2542,3,In the century-old statues that occupy Cape To...,795,TEDGlobal 2017,1503792000,1,Sethembile Msezane,Sethembile M

In [9]:
len(df_nona['main_speaker'].unique())

942

In [10]:
df_nona = df_nona.drop('Unnamed: 0', axis=1)

In [11]:
df_nona.head()

,comments,description,duration,event,film_date,languages,main_speaker,name,num_speaker,published_date,ratings,related_talks,speaker_occupation,tags,title,url,views,city,country,continent
0,4553,Sir Ken Robinson makes an entertaining and pro...,1164,TED2006,1140825600,60,Ken Robinson,Ken Robinson: Do schools kill creativity?,1,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 19645}, {...","[{'id': 865, 'hero': 'https://pe.tedcdn.com/im...",Author/educator,"['children', 'creativity', 'culture', 'dance',...",Do schools kill creativity?,https://www.ted.com/talks/ken_robinson_says_sc...,47227110,Liverpool,United Kingdom,Europe
1,265,With the same humor and humanity he exuded in ...,977,TED2006,1140825600,43,Al Gore,Al Gore: Averting the climate crisis,1,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 544}, {'i...","[{'id': 243, 'hero': 'https://pe.tedcdn.com/im...",Climate advocate,"['alternative energy', 'cars', 'climate change...",Averting the climate crisis,https://www.ted.com/talks/al_gore_on_averting_...,3200520,Washington,United States,Americas
2,200,"In an emotionally charged talk, MacArthur-winn...",1116,TED2006,1140912000,35,Majora Carter,Majora Carter: Greening the ghetto,1,1151367060,"[{'id': 3, 'name': 'Courageous', 'count': 760}...","[{'id': 1041, 'hero': 'https://pe.tedcdn.com/i...",Activist for environmental justice,"['MacArthur grant', 'activism', 'business', 'c...",Greening the ghetto,https://www.ted.com/talks/majora_carter_s_tale...,1697550,New York,United States,Americas
3,593,You've never seen data presented like this. Wi...,1190,TED2006,1140566400,48,Hans Rosling,Hans Rosling: The best stats you've ever seen,1,1151440680,"[{'id': 9, 'name': 'Ingenious', 'count': 3202}...","[{'id': 2056, 'hero': 'https://pe.tedcdn.com/i...",Global health expert; data visionary,"['Africa', 'Asia', 'Google', 'demo', 'economic...",The best stats you've ever seen,https://www.ted.com/talks/hans_rosling_shows_t...,12005869,Uppsala,Sweden,Europe
4,672,"Tony Robbins discusses the ""invisible forces"" ...",1305,TED2006,1138838400,36,Tony Robbins,Tony Robbins: Why we do what we do,1,1151440680,"[{'id': 7, 'name': 'Funny', 'count': 1102}, {'...","[{'id': 229, 'hero': 'https://pe.tedcdn.com/im...",Life coach; expert in leadership psychology,"['business', 'culture', 'entertainment', 'goal...",Why we do what we do,https://www.ted.com/talks/tony_robbins_asks_wh...,20685401,Los Angeles,United States,Americas


In [12]:
locations = pd.read_csv('cs448b-final-project/locations.csv')

In [13]:
locations.head()

,name,lat,lng
0,Bertrand Piccard,46.521827,6.632702
1,Nick Veasey,51.507446,-0.127765
2,Dan Buettner,44.949749,-93.093103
3,Romulus Whitaker,40.712728,-74.006015
4,Herbie Hancock,41.875562,-87.624421


In [14]:
merged_df = df_nona.merge(locations, left_on='main_speaker', right_on='name', how='left')

In [15]:
merged_df

,comments,description,duration,event,film_date,languages,main_speaker,name_x,num_speaker,published_date,...,tags,title,url,views,city,country,continent,name_y,lat,lng
0,4553,Sir Ken Robinson makes an entertaining and pro...,1164,TED2006,1140825600,60,Ken Robinson,Ken Robinson: Do schools kill creativity?,1,1151367060,...,"['children', 'creativity', 'culture', 'dance',...",Do schools kill creativity?,https://www.ted.com/talks/ken_robinson_says_sc...,47227110,Liverpool,United Kingdom,Europe,Ken Robinson,53.407199,-2.991680
1,265,With the same humor and humanity he exuded in ...,977,TED2006,1140825600,43,Al Gore,Al Gore: Averting the climate crisis,1,1151367060,...,"['alternative energy', 'cars', 'climate change...",Averting the climate crisis,https://www.ted.com/talks/al_gore_on_averting_...,3200520,Washington,United States,Americas,Al Gore,38.895037,-77.036543
2,200,"In an emotionally charged talk, MacArthur-winn...",1116,TED2006,1140912000,35,Majora Carter,Majora Carter: Greening the ghetto,1,1151367060,...,"['MacArthur grant', 'activism', 'business', 'c...",Greening the ghetto,https://www.ted.com/talks/majora_carter_s_tale...,1697550,New York,United States,Americas,Majora Carter,40.712728,-74.006015
3,593,You've never seen data presented like this. Wi...,1190,TED2006,1140566400,48,Hans Rosling,Hans Rosling: The best stats you've ever seen,1,1151440680,...,"['Africa', 'Asia', 'Google', 'demo', 'economic...",The best stats you've ever seen,https://www.ted.com/talks/hans_rosling_shows_t...,12005869,Uppsala,Sweden,Europe,Hans Rosling,59.858613,17.638744
4,672,"Tony Robbins discusses the ""invisible forces"" ...",1305,TED2006,1138838400,36,Tony Robbins,Tony Robbins: Why we do what we do,1,1151440680,...,"['business', 'culture', 'entertainment', 'goal...",Why we do what we do,https://www.ted.com/talks/tony_robbins_asks_wh...,20685401,Los Angeles,United States,Americas,Tony Robbins,34.053691,-118.242766
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1199,9,In a mind-bending talk that blurs the line bet...,663,TED2017,1492992000,2,Tomás Saraceno,Tomás Saraceno: Would you live in a floating ...,1,1504796048,...,"['TED en Español', 'art', 'creativity', 'desig...",Would you live in a floating city in the sky?,https://www.ted.com/talks/tomas_saraceno_would...,248411,San Miguel de Tucumán,Argentina,Americas,Tomás Saraceno,-26.830370,-65.203813
1200,7,"Each year, one in seven large corporations com...",808,TEDxZurich,1478822400,1,Alexander Wagner,Alexander Wagner: What really motivates people...,1,1505246451,...,"['TEDx', 'business', 'corruption', 'economics'...",What really motivates people to be honest in b...,https://www.ted.com/talks/alexander_wagner_wha...,621431,Berlin,Germany,Europe,Alexander Wagner,52.517037,13.388860
1201,12,Physics doesn't just happen in a fancy lab -- ...,957,TEDxManchester,1487462400,2,Helen Czerski,Helen Czerski: The fascinating physics of ever...,1,1505419202,...,"['TEDx', 'physics', 'play', 'science']",The fascinating physics of everyday life,https://www.ted.com/talks/helen_czerski_fun_ho...,656791,Manchester,United Kingdom,Europe,Helen Czerski,53.479489,-2.245115
1202,3,In the century-old statues that occupy Cape To...,795,TEDGlobal 2017,1503792000,1,Sethembile Msezane,Sethembile Msezane: Living sculptures that sta...,1,1505488093,...,"['Africa', 'activism', 'art', 'community', 'hi...",Living sculptures that stand for history's truths,https://www.ted.com/talks/sethembile_msezane_l...,542088,Umvoti Local Municipality,South Africa,Africa,Sethembile Msezane,-28.943694,30.470385


In [22]:
continent_df = merged_df[['main_speaker','views','lat', 'lng', 'city','country','continent']]

In [18]:
filtered_df.head()

,main_speaker,views,lat,lng,city,country
0,Ken Robinson,47227110,53.407199,-2.991680,Liverpool,United Kingdom
1,Al Gore,3200520,38.895037,-77.036543,Washington,United States
2,Majora Carter,1697550,40.712728,-74.006015,New York,United States
3,Hans Rosling,12005869,59.858613,17.638744,Uppsala,Sweden
4,Tony Robbins,20685401,34.053691,-118.242766,Los Angeles,United States


In [19]:
aggregated_df = filtered_df.groupby(['main_speaker', 'lat', 'lng','city','country']).agg(mean_views=('views', 'mean'), number_of_talks=('views', 'count')).reset_index()
aggregated_df['mean_views'] = aggregated_df['mean_views'].astype('int')
aggregated_df = aggregated_df.rename(columns={
    'main_speaker': 'name', 
    'mean_views': 'meanviews', 
    'number_of_talks': 'numtalks'
})

In [20]:
aggregated_df

,name,lat,lng,city,country,meanviews,numtalks
0,OK Go,32.745964,-96.997785,Grand Prairie,United States,1493866,1
1,Silk Road Ensemble,48.858890,2.320041,Paris,France,640734,1
2,Sofi Tukker,-24.308804,-54.748267,Katueté,Paraguay,815611,1
3,AJ Jacobs,40.712728,-74.006015,New York,United States,1622792,3
4,Aakash Odedra,52.479699,-1.902691,Birmingham,United Kingdom,817014,1
...,...,...,...,...,...,...,...
937,Zainab Salbi,33.306170,44.387221,Baghdad,Iraq,505716,1
938,Zak Ebrahim,40.441694,-79.990086,Pittsburgh,United States,4619628,1
939,Ziyah Gafić,43.851977,18.386687,Sarajevo,Bosnia and Herzegovina,1000194,1
940,Zubaida Bai,13.083694,80.270186,Chennai,India,801279,1


In [21]:
aggregated_df.to_csv('locations_and_views.csv', index=None)

In [ ]:
continent_df = merged_df[['main_speaker','views','lat', 'lng', 'city','country','continent']]

In [23]:
cont_aggr_df = continent_df.groupby(['main_speaker', 'lat', 'lng','city','country','continent']).agg(mean_views=('views', 'mean'), number_of_talks=('views', 'count')).reset_index()
cont_aggr_df['mean_views'] = cont_aggr_df['mean_views'].astype('int')
cont_aggr_df = cont_aggr_df.rename(columns={
    'main_speaker': 'name', 
    'mean_views': 'meanviews', 
    'number_of_talks': 'numtalks'
})

In [24]:
cont_aggr_df

,name,lat,lng,city,country,continent,meanviews,numtalks
0,OK Go,32.745964,-96.997785,Grand Prairie,United States,Americas,1493866,1
1,Silk Road Ensemble,48.858890,2.320041,Paris,France,Europe,640734,1
2,Sofi Tukker,-24.308804,-54.748267,Katueté,Paraguay,Americas,815611,1
3,AJ Jacobs,40.712728,-74.006015,New York,United States,Americas,1622792,3
4,Aakash Odedra,52.479699,-1.902691,Birmingham,United Kingdom,Europe,817014,1
...,...,...,...,...,...,...,...,...
937,Zainab Salbi,33.306170,44.387221,Baghdad,Iraq,Asia,505716,1
938,Zak Ebrahim,40.441694,-79.990086,Pittsburgh,United States,Americas,4619628,1
939,Ziyah Gafić,43.851977,18.386687,Sarajevo,Bosnia and Herzegovina,Europe,1000194,1
940,Zubaida Bai,13.083694,80.270186,Chennai,India,Asia,801279,1


In [25]:
agg_df = cont_aggr_df.groupby(['continent', 'city']).agg({'meanviews': 'sum', 'lat': 'first', 'lng': 'first'}).reset_index()
top_cities = agg_df.sort_values(['continent', 'meanviews'], ascending=[True, False]).groupby('continent').head(5)

In [26]:
top_cities

,continent,city,meanviews,lat,lng
7,Africa,Enugu,7308397,6.449983,7.500001
22,Africa,Pretoria,5589405,-25.745928,28.187910
0,Africa,Accra,4457871,5.557110,-0.201238
4,Africa,Bulawayo,4191678,-20.105395,28.542686
13,Africa,Johannesburg,3777325,-26.205000,28.049722
138,Americas,New York,166514399,40.712728,-74.006015
112,Americas,Los Angeles,64590486,34.053691,-118.242766
215,Americas,Washington,49856494,38.895037,-77.036543
173,Americas,San Francisco,49484015,37.779026,-122.419906
152,Americas,Philadelphia,36304858,39.943911,-75.183764


In [29]:
top_cities.to_csv('cs448b-final-project/top-cities-views.csv', index=None)

In [27]:
grouped = cont_aggr_df.groupby(['continent', 'city']).agg({'name': pd.Series.nunique, 'lat': 'first', 'lng': 'first'}).reset_index()

grouped.rename(columns={'name': 'num_speakers'}, inplace=True)

top_cities_speakers = grouped.sort_values(['continent', 'num_speakers'], ascending=[True, False]).groupby('continent').head(5)

In [28]:
top_cities_speakers

,continent,city,num_speakers,lat,lng
0,Africa,Accra,4,5.557110,-0.201238
13,Africa,Johannesburg,3,-26.205000,28.049722
21,Africa,Nairobi,3,-1.302615,36.828842
1,Africa,Addis Ababa,2,9.035829,38.752413
6,Africa,Cape Town,2,-33.928992,18.417396
138,Americas,New York,100,40.712728,-74.006015
112,Americas,Los Angeles,28,34.053691,-118.242766
55,Americas,Chicago,23,41.875562,-87.624421
215,Americas,Washington,19,38.895037,-77.036543
45,Americas,Boston,14,42.355433,-71.060511


In [30]:
top_cities_speakers.to_csv('cs448b-final-project/top-cities-numtalks.csv', index=None)